In [1]:
import pandas as pd
import sqlite3 as sql
import numpy as np
import matplotlib.pyplot as plt

In [4]:
db = 'chinook.db'
def run_query(q):
    with sql.connect(db) as conn:
        return pd.read_sql_query(q,conn)

In [13]:
q = """
select t1.track_id,t1.album_id,t2.track_id,t2.album_id from track t1
inner join track t2
on
t1.album_id == t2.album_id
where t1.track_id == 1158
"""
run_query(q).head(30)

,track_id,album_id,track_id,album_id
0,1158,91,1158,91
1,1158,91,1159,91
2,1158,91,1160,91
3,1158,91,1161,91
4,1158,91,1162,91
5,1158,91,1163,91
6,1158,91,1164,91
7,1158,91,1165,91
8,1158,91,1166,91
9,1158,91,1167,91


In [14]:
q = """
select t2.track_id from track t1
inner join track t2
on
t1.album_id == t2.album_id
where t1.track_id == 1158
"""
run_query(q).head(30)

,track_id
0,1158
1,1159
2,1160
3,1161
4,1162
5,1163
6,1164
7,1165
8,1166
9,1167


In [17]:
q = """
select il.track_id from invoice_line il
where il.Invoice_id = 1
"""
run_query(q)

,track_id
0,1158
1,1159
2,1160
3,1161
4,1162
5,1163
6,1164
7,1165
8,1166
9,1167


In [18]:
q = """
select t2.track_id from track t1
inner join track t2
on
t1.album_id == t2.album_id
where t1.track_id == 1158

except 

select il.track_id from invoice_line il
where il.Invoice_id = 1
"""
run_query(q).head(30)

,track_id


In [27]:
q = """

with invoice_data as
(select invoice_id,track_id from invoice_line
group by 1
having invoice_id =1)

select * from invoice_data
"""
run_query(q)

,invoice_id,track_id
0,1,1158


In [29]:
q = """

with invoice_data as
(select invoice_id,track_id from invoice_line
group by 1
having invoice_id =1)

select invoice_id,
case
    when(
        select t2.track_id from track t1
    inner join track t2
    on
    t1.album_id == t2.album_id
    where t1.track_id == invd.track_id

    except 

    select il.track_id from invoice_line il
    where il.Invoice_id = invd.invoice_id
        ) is null
        
    then "Yes"
    else "No"
end as Purchased_Album


from invoice_data invd
"""
run_query(q)

,invoice_id,Purchased_Album
0,1,Yes


In [30]:
q = """

with invoice_data as
(select invoice_id,track_id from invoice_line
group by 1
having invoice_id =1)

select invoice_id,
case
    when(
        select t2.track_id from track t1
        inner join track t2
        on
        t1.album_id == t2.album_id
        where t1.track_id == invd.track_id

        except 

        select il.track_id from invoice_line il
        where il.Invoice_id = invd.invoice_id
        ) is null
        
        AND
        (
        select il.track_id from invoice_line il
        where il.Invoice_id = invd.invoice_id
        
        except
        
        select t2.track_id from track t1
        inner join track t2
        on
        t1.album_id == t2.album_id
        where t1.track_id == invd.track_id
        ) is null
        
        
    then "Yes"
    else "No"
end as Purchased_Album


from invoice_data invd
"""
run_query(q)

,invoice_id,Purchased_Album
0,1,Yes


In [38]:
q = """

with invoice_data as
(select invoice_id,track_id from invoice_line
group by 1
),

album_purchased as
(select invoice_id,
case
    when(
        select t2.track_id from track t1
        inner join track t2
        on
        t1.album_id == t2.album_id
        where t1.track_id == invd.track_id

        except 

        select il.track_id from invoice_line il
        where il.Invoice_id = invd.invoice_id
        ) is null
        
        AND
        (
        select il.track_id from invoice_line il
        where il.Invoice_id = invd.invoice_id
        
        except
        
        select t2.track_id from track t1
        inner join track t2
        on
        t1.album_id == t2.album_id
        where t1.track_id == invd.track_id
        ) is null
        
        
    then "Yes"
    else "No"
end as Purchased_Album

from invoice_data invd)

select Purchased_Album,count(invoice_id) as no_of_invoices,
    cast(count(invoice_id) as float) * 100/(select count(invoice_id) from album_purchased) as percentage
from album_purchased
group by 1
"""
run_query(q)

,Purchased_Album,no_of_invoices,percentage
0,No,500,81.433225
1,Yes,114,18.566775
